# Sentences as Circuits

### Import stuff

In [ ]:
from discopy.rigid import Ty
from discopy.grammar.pregroup import Word
from discopy.rigid import Diagram, Id, Cup
import yaml
import random
from math import pi
from qnlpws_utils.parse import *
from qnlpws_utils.circuits import *
import ipywidgets as ipw
import pickle
from pytket.extensions.qiskit import AerBackend, tk_to_qiskit
from pytket.extensions.qsharp import QsharpSimulatorBackend, tk_to_qsharp
import numpy as np
from geneticalgorithm import geneticalgorithm as ga
import noisyopt
from time import time

In [ ]:
with open("data/words.yaml") as f:
    word_data = yaml.safe_load(f)

In [ ]:
s, n = Ty('s'), Ty('n')

### Create sets of verbs and nouns and sentences

- These are simple "pseudo-sentences" used by Hoffmann in his thesis, and are stored in two parts:
    - A string containing the verb and noun
    - An integer (0 or 1) representing the plausibility score of a sentence

In [ ]:
dataset, nouns, verbs = parse_dummy_sentences(word_data["sub_sentences"])

## Create representation of our words

Here, words are added to a dictionary, `vocab`, along with their structure in the DisCoCat model.

In [ ]:
ambiguous_verbs = ["file"] 

# nouns
vocab = Vocab({noun: Word(noun, n) for noun in nouns},
              {verb: Word(verb, s @ n.l) for verb in verbs if verb not in ambiguous_verbs},
              {amb: Word(amb, s @ n.l) for amb in ambiguous_verbs}
)

## Prepare and parse dataset
### Define a Grammar
Here, we define the input pseudo-sentence as consisting of a verb and a left-adjoint-noun and noun pair and parse the sentences accordingly.

In [ ]:
grammar = Id(s) @ Cup(n.l, n)

In [ ]:
sentences, plausability, parsing = parse_with_grammar(dataset, grammar, vocab)

In [ ]:
len(sentences)

### View Sentences

The result of this parsing can be seen below. Use the dropdown menu to select which sentence you want to view. Note: for some reason, the dropdown list works on some occasions but not on others. 

In [ ]:
dc = ipw.Dropdown(options=list(zip(sentences, plausability)))
@ipw.interact(p=dc)
def view_psentences(p):
    print(dc.label," plausability: ", p)
    parsing[dc.label].draw(draw_type_labels=True)

## Creating Circuits
### Creating initial parameters
The initial values of the parameters for nouns and ambiguous verbs are generated randomly according to a uniform distribution between 0 and 1 using the function create_params.

In [ ]:
params = \
    create_params(nouns, 
                  ambiguous_verbs, 
                  vocab)

### Generating the circuits

Circuits are created using the function $F$. For the curious, the definition is shown below. You can choose the backend of your choice, but the Q# backend does not include a way of displaying the circuit.

```Python
def F(vocab,
      params,
      n_qubits_ansatz=1):
    ar1 = {vocab.noun[noun]:noun_ansatz(params.noun[noun]) for noun in vocab.noun}
    ar2 = {vocab.unamb_verb[verb]:un_amb_verb_ansatz(params.unamb_verb[verb]) for verb in vocab.unamb_verb}
    ar3 = {vocab.amb_verb[verb]:amb_verb_ansatz(params.amb_verb[verb]) for verb in vocab.amb_verb}
    ar = {**ar1, **ar2, **ar3}

    return CircuitFunctor(
        ob = {s: qubit ** 0, n: qubit ** n_qubits_ansatz},
        ar = ar)
```

In [ ]:
circuits = {s:F(vocab,params)(parsing[s]) for s in sentences}

In [ ]:
qiskit_backend = AerBackend()
qsharp_backend = QsharpSimulatorBackend()

### Choose a sentence

In [ ]:
ds = ipw.Dropdown(options=circuits.items(), description="sentence")
ds

### Choose the backend
Note: Q# backend can't display circuits

In [ ]:
dbe = ipw.Dropdown(options=["qiskit", "qsharp"], description="backend")
dbe

### Create the circuit

In [ ]:
circuit = ds.value
if dbe.value == "qsharp":
    backend = qsharp_backend
else:
    backend = qiskit_backend

### Render the circuit

In [ ]:
def render_circuit(circuit, backend):
    tk_circ = circuit.to_tk()
    print("{}:\n{}\n".format(tk_circ, '\n'.join(map(str, tk_circ))))
    print("post selection:\n{}\n".format(tk_circ.post_selection))
    print("scalar:\n{}\n".format(tk_circ.scalar))
    print("qiskit circuit:")
    if isinstance(backend, AerBackend):
        tk_to_qiskit(tk_circ).draw(output="mpl")
    else:
        print("Circuit drawing not implemented for this backend")

In [ ]:
render_circuit(circuit, backend)

## Optimizing the parameters
### Step 1: [Genetic algorithm](https://en.wikipedia.org/wiki/Genetic_algorithm)
In this section, a Genetic Algorithm is applied to optimize the parameters for the representation of words as qubits. A genetic algorithm is inspired by natural selection in evolution and uses random mutations in parameters, letting the inadequate mutations die off to hopefully end up with a superior parameter set. These are expensive, however, and the rate of decrease in the error slows after a few iterations, so here its use is limited to 25 iterations. 

In [ ]:
def reshape_data(params_np, vocab, n_noun_params = 2, n_amb_params = 1):
    ''' Converts numpy array of parameters back to dictionary
    '''
    n_nouns = len(vocab.noun)
    n_amb_verbs = len(vocab.amb_verb)
    n_unamb_verbs = len(vocab.unamb_verb)

    params_nouns_np = params_np[:n_nouns*n_noun_params].reshape((n_nouns,n_noun_params))
    params_amb_verbs_np = params_np[n_nouns*n_noun_params:].reshape((n_amb_verbs,n_amb_params))

    params_nouns = {word: params_nouns_np[i].tolist() for i, word in enumerate(vocab.noun.keys())}
    params_amb_verbs = {word: params_amb_verbs_np[i].tolist() for i, word in enumerate(vocab.amb_verb)}

    return params_nouns, params_amb_verbs

In [ ]:
def evaluate(params_nouns, params_unamb_verbs, params_amb_verbs, 
             sentences, backend=AerBackend(), n_shots=2**10, seed=0):
    global vocab
    circuits = [F(vocab, Params(params_nouns, params_unamb_verbs, params_amb_verbs))(parsing[sent]) for sent in sentences]
    results = [Circuit.eval(
                circuit,
                backend=backend,
                n_shots=n_shots,
                seed=seed,
                compilation=backend.default_compilation_pass(2)) for circuit in circuits]
    tensors = [np.abs(result.array)[0] for result in results]
    return tensors

In [ ]:
def loss(params_np):
    global circuits
    global vocab
    global plausability
    global backend
    global params
    global sentences
    # convert np to dict
    params_nouns, params_amb_verbs = reshape_data(params_np,
                                                 vocab,
                                                  n_noun_params = 2, n_amb_params = 1 )
    return np.mean(np.array([
        (plausability[i] - scalar) ** 2
        for i, scalar in enumerate(evaluate(params_nouns, 
                                            params.unamb_verb, 
                                            params_amb_verbs, 
                                            sentences,
                                            backend))]))

In [ ]:
def get_model(func,
              vocab,
              n_qubits_ansatz = 1,
               n_noun_params = 2,
               n_amb_params = 1,
               algorithm_param = None):
    if algorithm_param == None:
        algorithm_param = {
            'max_num_iteration': 25,\
            'population_size':5,\
            'mutation_probability':0.1,\
            'elit_ratio': 0.01,\
            'crossover_probability': 0.5,\
            'parents_portion': 0.3,\
            'crossover_type':'uniform',\
            'max_iteration_without_improv':None
        }

    dimension = n_noun_params*len(vocab.noun) + n_amb_params*len(vocab.amb_verb)
    varbound=np.array([[0,1]]*dimension)

    return ga(function=func, dimension=dimension, 
             variable_type='real',
             variable_boundaries=varbound, 
             algorithm_parameters=algorithm_param, 
             function_timeout=100)

In [ ]:
model = get_model(loss, vocab)

In [ ]:
model.run()

### Step 2: noisyopt

noisyopt uses an optimization method that simulates the constraints of optimization problems on a quantum-device. Such algorithms have been proven to work with QNLP on noisy-intermediate-stage quantum (NISQ) computers (Meichanetzidis et al.)

In [ ]:
i, start = 0, time()
def callback(loss):
    global i
    i += 1
    print("Epoch {} ({:.0f} seconds since start): {}".format(i, time() - start, loss))

result = noisyopt.minimizeSPSA(
    loss, model.best_variable, paired=False, callback=callback, niter=200, a=0.2, c=0.1)

print("Best loss: ", result.fun)

In [ ]:
def loss2(params_np, 
          vocab=vocab,
          params=params,
          sentences=sentences, 
          plausability=plausability):

    # convert back to dict
    params_nouns, params_amb_verbs = reshape_data(params_np, vocab)

    return  {sentences[i]:(plausability[i], round(scalar,4))
        for i, scalar in enumerate(evaluate(params_nouns, 
                                            params.unamb_verb, 
                                            params_amb_verbs, 
                                            sentences))}

In [ ]:
results = loss2(result.x, vocab)
results

### Similarity

The images below show the similarity between this model's interpretation of a sentence's plausibility and the true value. The top image is for a model with a maximum loss of 0.0414 and the bottom is for a model with loss of 0.0153.
<table>
    <tr>
        <td><img src = amb_results0414.png width = 512></td>
        <td><img src = amb_results0153.png width = 512></td>
    </tr>
    <tr>
        <td align = center> loss = 0.0414 </td>
        <td align = center> loss = 0.0153 </td>
    </tr>
</table>

`high` indicates that the verb is not `file` and the sentences is assigned a plausibility of 1.
`low` idicates the non-`file` verbs with plausibility 0.
`amb` indicates that the verb is `file`.